In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

data = pd.read_pickle("../data/df_preproc.pk")
hashtags_vocab = reduce((lambda x, y: set(x).union(y)), data.Hashtags)
mentions_vocab = reduce((lambda x, y: set(x).union(y)), data.Mentions)

_embed = None

In [2]:
print(data)

     Avis Entreprise                                              Tweet  \
0000  neu        apl                20 min line apple store short pump.   
0001  irr        msf  Nueva tecnología convierte cualquier superfici...   
0002  neu        ggl  Some people should not post replies in Google+...   
0003  neg        apl  I know a few others having same issue RT Joelp...   
0004  neg        msf  Microsoft - We put the ""backwards"" into back...   
...   ...        ...                                                ...   
4168  neg        apl  fuck this see you hoes  work WeakTwip Munnns a...   
4169  neg        msf  Microsoft, Adobe lose $13.5bn to piracy: Repor...   
4170  neu        twt  I tried to explain why you would do ""The Twit...   
4171  neg        apl  Installed io5 - fine on ipad but wiped wife's ...   
4172  neg        msf  microsoft careers site is giving errors for an...   

                            Hashtags                   Mentions Language  \
0000                   

In [144]:
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras.layers import Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


def embedding_model(vocab, max_len, max_features, out_dim):
    vectorize_layer = TextVectorization(
        max_tokens=max_features,
        output_sequence_length=max_len,
        output_mode="int",
        # split=None,
        vocabulary=list(vocab),
    )
    return tf.keras.Sequential(
        layers=[vectorize_layer, Embedding(max_features + 1, out_dim), Flatten()]
    )


max_hashtags = 5
max_mentions = 5

_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
_hashtag_embed = embedding_model(hashtags_vocab, 8, 2000, 16)
_mention_embed = embedding_model(mentions_vocab, 8, 1500, 16)

FileNotFoundError: Op type not registered 'SentencepieceOp' in binary running on alones-MacBook-Pro.local. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
 If trying to load on a different device from the computational device, consider using setting the `experimental_io_device` option on tf.saved_model.LoadOptions to the io_device such as '/job:localhost'.

<IPython.core.display.Javascript object>

In [47]:
print(_hashtag_embed.predict([["google apple"], ["app quote"]]).shape)
print(_mention_embed.predict([["google apple"], ["tesla"]]).shape)
# print(_embed.predict(data.Tweet[10:15]).shape)

embeddings = pd.DataFrame(
    {
        "avis": data.Avis.astype("category").cat.codes,
        "language": data.Language.astype("category").cat.codes,
    }
)
# embeddings.to_pickle("../data/embeddings.pkl")

(2, 128)
(2, 128)


<IPython.core.display.Javascript object>

In [109]:
data["Irr"] = data.Avis.str.fullmatch("irr").astype("int")
data.Irr

0000    0
0001    1
0002    0
0003    0
0004    0
       ..
4168    0
4169    0
4170    0
4171    0
4172    0
Name: Irr, Length: 4173, dtype: int64

<IPython.core.display.Javascript object>

In [ ]:
tweets = np.hstack(
    [
        _embed(data.Tweet).numpy(),
        _mention_embed.predict(data.Mentions.apply(lambda x: " ".join(x))),
        _hashtag_embed.predict(data.Hashtags.apply(lambda x: " ".join(x))),
    ]
)

In [133]:
from thinc.api import (
    Model,
    chain,
    add,
    concatenate,
    clone,
    strings2arrays,
    array_getitem,
    with_array,
    Embed,
    HashEmbed,
    expand_window,
    Linear,
    Relu,
    Softmax,
    Dropout,
    Adam,
    warmup_linear,
    TensorFlowWrapper,
)
from thinc.types import List1d, Array2d
from thinc.util import to_categorical

<IPython.core.display.Javascript object>

In [ ]:
n_hidden = 64
dropout = 0.2


def Hidden(n_hidden=10, dropout=0.2):
    return chain(Relu(nO=n_hidden), Dropout(dropout))


with Model.define_operators({">>": chain, "|": concatenate, "+": add, "**": clone}):
    model = strings2arrays() >> Hidden(n_hidden, dropout) ** 2 >> Softmax()

In [ ]:
# model.initialize(X=data.Hashtags.to_numpy(), Y=data.Avis.cat.codes)

In [ ]:
data.Avis = data.Avis.astype("category")

In [26]:
from syntok.tokenizer import Tokenizer


def tokenize_texts(texts):
    tok = Tokenizer()
    return [[token.value for token in tok.tokenize(text)] for text in texts]


import ml_datasets
import numpy


def load_data():
    train_data, dev_data = ml_datasets.dbpedia(train_limit=2000, dev_limit=2000)
    train_texts, train_cats = zip(*train_data)
    dev_texts, dev_cats = zip(*dev_data)
    unique_cats = list(numpy.unique(numpy.concatenate((train_cats, dev_cats))))
    nr_class = len(unique_cats)
    print(f"{len(train_data)} training / {len(dev_data)} dev\n{nr_class} classes")

    train_y = numpy.zeros((len(train_cats), nr_class), dtype="f")
    for i, cat in enumerate(train_cats):
        train_y[i][unique_cats.index(cat)] = 1
    dev_y = numpy.zeros((len(dev_cats), nr_class), dtype="f")
    for i, cat in enumerate(dev_cats):
        dev_y[i][unique_cats.index(cat)] = 1

    train_tokenized = tokenize_texts(train_texts)
    dev_tokenized = tokenize_texts(dev_texts)
    # Generate simple vocab mapping, <unk> is 0
    vocab = {}
    count_id = 1
    for text in train_tokenized:
        for token in text:
            if token not in vocab:
                vocab[token] = count_id
                count_id += 1
    # Map texts using vocab
    train_X = []
    for text in train_tokenized:
        train_X.append(numpy.array([vocab.get(t, 0) for t in text]))
    dev_X = []
    for text in dev_tokenized:
        dev_X.append(numpy.array([vocab.get(t, 0) for t in text]))
    return (train_X, train_y), (dev_X, dev_y), vocab

<IPython.core.display.Javascript object>

In [27]:
(train_X, train_y), (dev_X, dev_y), vocab = load_data()

2000 training / 2000 dev
14 classes


<IPython.core.display.Javascript object>

In [143]:
len(train_X)

2000

<IPython.core.display.Javascript object>

In [29]:
from typing import List
import thinc
from thinc.api import Model, chain, list2ragged, with_array, reduce_mean, Softmax
from thinc.types import Array2d


@thinc.registry.layers("EmbedPoolTextcat.v1")
def EmbedPoolTextcat(embed: Model[Array2d, Array2d]) -> Model[List[Array2d], Array2d]:
    with Model.define_operators({">>": chain}):
        model = with_array(embed) >> list2ragged() >> reduce_mean() >> Softmax()
    model.set_ref("embed", embed)
    return model

<IPython.core.display.Javascript object>

In [30]:
CONFIG = """
[hyper_params]
width = 64

[model]
@layers = "EmbedPoolTextcat.v1"

[model.embed]
@layers = "Embed.v1"
nO = ${hyper_params:width}

[optimizer]
@optimizers = "Adam.v1"
learn_rate = 0.001

[training]
batch_size = 8
n_iter = 10
"""

<IPython.core.display.Javascript object>

In [31]:
from thinc.api import registry, Config

C = registry.resolve(Config().from_str(CONFIG))
C

{'hyper_params': {'width': 64},
 'model': <thinc.model.Model at 0x15fca7140>,
 'optimizer': <thinc.optimizers.Optimizer at 0x15dcbc860>,
 'training': {'batch_size': 8, 'n_iter': 10}}

<IPython.core.display.Javascript object>

In [32]:
batch_size = C["training"]["batch_size"]
optimizer = C["optimizer"]
model = C["model"]
model.get_ref("embed").set_dim("nV", len(vocab) + 1)

model.initialize(X=train_X, Y=train_y)

<IPython.core.display.Javascript object>

In [88]:
def evaluate_model(model, dev_X, dev_Y, batch_size):
    correct = 0.0
    total = 0.0
    for X, Y in model.ops.multibatch(batch_size, dev_X, dev_Y):
        Yh = model.predict(X)
        for j in range(len(Yh)):
            correct += Yh[j].argmax(axis=0) == Y[j].argmax(axis=0)
        total += len(Y)
    return float(correct / total)

<IPython.core.display.Javascript object>

In [34]:
from thinc.api import fix_random_seed
from tqdm.notebook import tqdm

fix_random_seed(0)
for n in range(C["training"]["n_iter"]):
    loss = 0.0
    batches = model.ops.multibatch(batch_size, train_X, train_y, shuffle=True)
    for X, Y in tqdm(batches, leave=False):
        Yh, backprop = model.begin_update(X)
        d_loss = []
        for i in range(len(Yh)):
            d_loss.append(Yh[i] - Y[i])
            loss += ((Yh[i] - Y[i]) ** 2).sum()
        backprop(numpy.array(d_loss))
        model.finish_update(optimizer)
    score = evaluate_model(model, dev_X, dev_y, batch_size)
    print(f"{n}\t{loss:.2f}\t{score:.3f}")

  0%|          | 0/250 [00:00<?, ?it/s]

0	1838.10	0.459


  0%|          | 0/250 [00:00<?, ?it/s]

1	1617.94	0.658


  0%|          | 0/250 [00:00<?, ?it/s]

...

  0%|          | 0/250 [00:00<?, ?it/s]

8	83.55	0.928


  0%|          | 0/250 [00:00<?, ?it/s]

9	55.16	0.931


<IPython.core.display.Javascript object>

In [141]:
from thinc.api import (
    Model,
    chain,
    add,
    concatenate,
    clone,
    strings2arrays,
    array_getitem,
    residual,
    reduce_max,
    reduce_mean,
    with_array,
    Embed,
    HashEmbed,
    expand_window,
    Linear,
    Relu,
    Softmax,
    Dropout,
    Maxout,
    Adam,
    warmup_linear,
    TensorFlowWrapper,
)
from thinc.types import List1d, Array2d

(x, y, dx, dy) = (
    tweets,
    to_categorical(embeddings.avis),
    tweets[3101:],
    to_categorical(embeddings.avis[3101:]),
)

with Model.define_operators({">>": chain, "|": concatenate, "+": add, "**": clone}):
    model = reduce_mean() >> (Linear() >> Relu() >> Dropout(0.5)) ** 2 >> Softmax()
model.initialize(X=x, Y=y)

for n in range(20):
    loss = 0.0
    batches = model.ops.multibatch(batch_size, x, y, shuffle=True)
    for X, Y in tqdm(batches, leave=False):
        Yh, backprop = model.begin_update(X)
        d_loss = []
        for i in range(len(Yh)):
            d_loss.append(Yh[i] - Y[i])
            loss += ((Yh[i] - Y[i]) ** 2).sum()
        backprop(numpy.array(d_loss))
        model.finish_update(optimizer)
    score = evaluate_model(model, dx, dy, batch_size)
    print(f"{n}\t{loss:.2f}\t{score:.3f}")

AttributeError: 'numpy.ndarray' object has no attribute 'lengths'

<IPython.core.display.Javascript object>

In [121]:
tweets.shape

(4173, 384)

<IPython.core.display.Javascript object>